In [1]:
import pandas as pd
import pprint

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken

import os
from dotenv import load_dotenv

load_dotenv()
os.getenv('OPENAI_API_KEY')

client = AsyncOpenAI()
model = "gpt-4o-mini"

In [2]:
system_prompt = """
    L'utilisateur va fournir un extrait de 2 minutes d'une émission de télévision ou de radio.
    Le transcript ne contiendra pas de ponctuation et peut être de qualité médiocre (vocabulaire incorrect, mauvais découpage du texte). 
    La transcription est phonétique. Lorsque la phrase n'a pas de sens, reformule en prenant compte de la phonétique pour que le texte final soit en français correct.  
    
    Exemple :
    Transcription : 'elles doivent donc être renouvelés et sané col débuts en europe'
    Correction : 'Elles doivent donc être renouvelées, et ce n'est que le début en Europe.'
"""
encoding = tiktoken.encoding_for_model(model)
prompt_tokens = len(encoding.encode(system_prompt))

async def correct_text(
        text: str, system_prompt: str = system_prompt, prompt_tokens: str = prompt_tokens, tpm_limit: int = 2e6
    ):
    text_tokens = len(encoding.encode(text))
    total_tokens = prompt_tokens + text_tokens
    wait_time_s = 60 * total_tokens / tpm_limit
    await asyncio.sleep(wait_time_s)
    
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

async def run_correction(texts: list[str]):
    semaphore = asyncio.Semaphore(10)
    
    async def bounded_detect_claim(text):
        async with semaphore:
            return await correct_text(text)
    
    # Use tqdm_asyncio.gather with the bounded_detect_claim function
    correction = await tqdm.gather(*[bounded_detect_claim(text) for text in texts])
    return correction

#correction = await run_correction(df["text"])
#df['corrected_text'] = correction

In [54]:
pprint.pprint("tout mon dentiste pas parler d'un dentifrice protégeant ses astronautes qui réduit les saignements occasionne arthrose en classique le <unk> le dentifrice artero dont classique réduit de soixante douze pourcent les saignements occasionnels des gencives nouveau retrouvons fraîcheur intense l'efficacité la fraîcheur en plus artero dont jean civet solutions découvrez des lignes dynamiques un intérieur au design raffiné avec feu mais elle étonne encore mieux seulement vingt-trois kilomètres au compteur et beaucoup moins cher que la neuf et vous en avez beaucoup comme sacha aramisauto toutes celle cela là oui et des centaines des d'centaines autres d'autres jusqu'à trente pour cent moins chers que leurs valeurs neuf trouvez la vôtre sur aramisauto point com aramisauto authentique pastèque sans pépins bouquet la star gourmet de l'été un monde de possibilités dont vous seul connaissait les limites cnews la liberté d'expression n'a jamais fait autant parler pourquoi l'ordinaire devrait être la norme vous méritez plus méritez plus <unk> c'est votre mot nouveau lexus lbx des trois cent quarante-neuf euros par mois entretien cl <unk> on a la découverte d'une nouvelle espèce le mégot des villes à peine jeté sur le trottoir commence son grand voyage voyage échappant japon de justesse un agent de propreté est aidée par le vent la pluie ils se retrouvent dans les échos sa route va être parsemé d'embûches avant qui ne dérive pour atteindre sa destination finale l'océan entourés de milliers d'autres déchets venu comme lui de la terre il va se décomposer en polluants à lui tout seul des centaines de litres d'eau pierre <unk> <unk> <unk>")

("tout mon dentiste pas parler d'un dentifrice protégeant ses astronautes qui "
 'réduit les saignements occasionne arthrose en classique le <unk> le '
 'dentifrice artero dont classique réduit de soixante douze pourcent les '
 'saignements occasionnels des gencives nouveau retrouvons fraîcheur intense '
 "l'efficacité la fraîcheur en plus artero dont jean civet solutions découvrez "
 'des lignes dynamiques un intérieur au design raffiné avec feu mais elle '
 'étonne encore mieux seulement vingt-trois kilomètres au compteur et beaucoup '
 'moins cher que la neuf et vous en avez beaucoup comme sacha aramisauto '
 "toutes celle cela là oui et des centaines des d'centaines autres d'autres "
 "jusqu'à trente pour cent moins chers que leurs valeurs neuf trouvez la vôtre "
 'sur aramisauto point com aramisauto authentique pastèque sans pépins bouquet '
 "la star gourmet de l'été un monde de possibilités dont vous seul connaissait "
 "les limites cnews la liberté d'expression n'a jamais fait 

In [57]:
txt_with_advertisement = ["tout mon dentiste pas parler d'un dentifrice protégeant ses astronautes qui réduit les saignements occasionne arthrose en classique le <unk> le dentifrice artero dont classique réduit de soixante douze pourcent les saignements occasionnels des gencives nouveau retrouvons fraîcheur intense l'efficacité la fraîcheur en plus artero dont jean civet solutions découvrez des lignes dynamiques un intérieur au design raffiné avec feu mais elle étonne encore mieux seulement vingt-trois kilomètres au compteur et beaucoup moins cher que la neuf et vous en avez beaucoup comme sacha aramisauto toutes celle cela là oui et des centaines des d'centaines autres d'autres jusqu'à trente pour cent moins chers que leurs valeurs neuf trouvez la vôtre sur aramisauto point com aramisauto authentique pastèque sans pépins bouquet la star gourmet de l'été un monde de possibilités dont vous seul connaissait les limites cnews la liberté d'expression n'a jamais fait autant parler pourquoi l'ordinaire devrait être la norme vous méritez plus méritez plus <unk> c'est votre mot nouveau lexus lbx des trois cent quarante-neuf euros par mois entretien cl <unk> on a la découverte d'une nouvelle espèce le mégot des villes à peine jeté sur le trottoir commence son grand voyage voyage échappant japon de justesse un agent de propreté est aidée par le vent la pluie ils se retrouvent dans les échos sa route va être parsemé d'embûches avant qui ne dérive pour atteindre sa destination finale l'océan entourés de milliers d'autres déchets venu comme lui de la terre il va se décomposer en polluants à lui tout seul des centaines de litres d'eau pierre <unk> <unk> <unk>"]
correction = await run_correction(txt_with_advertisement)
pprint.pprint(correction)

["Tout mon dentiste parle d'un dentifrice protégeant ses utilisateurs, qui "
 "réduit les saignements occasionnels des gencives. C'est le dentifrice "
 'Artero, qui réduit de soixante-douze pour cent les saignements occasionnels. '
 'Nouveau, il offre une fraîcheur intense et une efficacité accrue. Découvrez '
 'des lignes dynamiques et un intérieur au design raffiné avec la nouvelle '
 'voiture, qui étonne encore mieux, seulement vingt-trois kilomètres au '
 'compteur et beaucoup moins cher que la version neuve. Vous en avez beaucoup, '
 "comme Sacha chez Aramisauto. Il y a des centaines d'autres modèles jusqu'à "
 'trente pour cent moins chers que leur valeur neuve. Trouvez la vôtre sur '
 'aramisauto.com.\n'
 '\n'
 "Aramisauto, authentique, c'est comme une pastèque sans pépins, un bouquet de "
 'possibilités dont vous seul connaissez les limites. CNews, la liberté '
 "d'expression n'a jamais fait autant parler. Pourquoi l'ordinaire devrait-il "
 "être la norme ? Vous méritez plus. C